# Randomized Algorithm for Nonnegative Convex Externalities

In [1]:
import numpy as np
import pandas as pd
import random
from scipy import stats

### 1. Data Initialization

- Assume that the optimal solution of the concave problem (5) is received as a numpy array of size (n*m) where n is the number of agents and m is the number of items. 
<br><br>
- Initialize the posibility matrix p

In [2]:
# num_agents : Number of agents
# num_items  : Number of items
# x          : Solution vector of the convex problem
# p          : Matrix representing possibilities that theta_i will be greater than x[i,j] for each component of matrix x

num_agents = 5
num_items = 3

x = np.mat([1,0.25,0,0.33,0.6,0,0.5,0.4,1,0,0.56,0.64,1,0.4,0.9])
x = np.reshape(x, (num_agents,num_items), order = 'F')
p = 1-x

print(x)

[[1.   0.   0.56]
 [0.25 0.5  0.64]
 [0.   0.4  1.  ]
 [0.33 1.   0.4 ]
 [0.6  0.   0.9 ]]


### 2. Algorithm Stage 1

- Round columns of matrix **$x$** independently
<br><br>
- For each $i \in [m]$ pick an independent uniform random variable $\theta_{i} \in [0,1]$, and for $j \in [n]$ let $x_{ji}^{\theta_{i}} = 1$ if $x_{ji} \geq \theta_{i}$, and $x_{ji}^{\theta_{i}} = 0$ otherwise. Moreover, let $x^{\theta} = [x_{1}^{\theta_{1}}|...|x_{m}^{\theta_{m}}]$ be the binary $n \times m$ random matrix obtained after this stage of rounding

In [5]:
# m : Number of items
# n : Number of agents
# theta : numpy array storing the value of theta_i for each row i of the matrix x

m = num_items
n = num_agents
theta = np.zeros(shape = m)

x_theta = np.copy(x.T) # copy the transposed version of matrix x

# store theta_i for each for of the matrix x
for i in np.arange(m):
    theta[i] = float(np.random.uniform(0,1,1))
    
# store binary values into matrix x_theta depend on the value of theta and x
for i in np.arange(m):
    x_theta[i][x_theta[i] >= theta[i]] = 1
    x_theta[i][x_theta[i] < theta[i]] = 0
            
x_theta = x_theta.T # transpose x_theta again

print(theta)

[0.19787958 0.87409985 0.3100013 ]


In [6]:
# check all matrices x, x_theta, and p

print(x)
print(x_theta)
print(p)

[[1.   0.   0.56]
 [0.25 0.5  0.64]
 [0.   0.4  1.  ]
 [0.33 1.   0.4 ]
 [0.6  0.   0.9 ]]
[[1. 0. 1.]
 [1. 0. 1.]
 [0. 0. 1.]
 [1. 1. 1.]
 [1. 0. 1.]]
[[0.   1.   0.44]
 [0.75 0.5  0.36]
 [1.   0.6  0.  ]
 [0.67 0.   0.6 ]
 [0.4  1.   0.1 ]]


### 3. Algorithm Stage 2

- Round $x^{\theta}$ to **$\hat{x}$** by separately applying the fair contention resolution to every row of
$x^{\theta}$

In [7]:
# A : numpy array storing the cardinality of set A_i for each row i as an integer
# r : output probability matrix of the fair contention resolution

A = np.sum(x_theta, axis=1)
r = np.zeros(shape = x_theta.shape)

for i in range(len(r)):
    # if the cardinality of A_i is 1 or 0, the i th row of r is equal to the i th row of x_theta
    if A[i] <= 1:
        r[i] = x_theta[i]
    else:
        for j in range(len(r[i])):
            sigma = np.sum(p[i,x_theta[i]==1])/(A[i]-1) + np.sum(p[i,x_theta[i]==0])/(A[i])
            if x_theta[i,j] == 1 : #if agent j is included in set A, subtract p[i,j]/(A[i] - 1) from sigma
                sigma -= p[i,j]/(A[i] - 1)
            else:
                sigma = 0
            r[i,j]=(1/np.sum(p[i]))*sigma
    print(np.sum(p[i]))

print(r)



1.44
1.6099999999999999
1.6
1.27
1.5
[[0.65277778 0.         0.34722222]
 [0.37888199 0.         0.62111801]
 [0.         0.         1.        ]
 [0.23622047 0.5        0.26377953]
 [0.4        0.         0.6       ]]


In [8]:
for i in range(r.shape[0]):
    xk = np.arange(len(r[i]))
    if np.sum(r[i]) == 0:
        r[i] = 1/len(r[i])
    pk = r[i]
    dist = stats.rv_discrete(values=(xk,pk))
    r[i] = 0
    r[i][dist.rvs(size=1)[0]] = 1
    
print(r)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
